# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [ ]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("max_columns",None)

# Olist packages
from olist.data import Olist
from olist.review import Review
from olist.order import Order
from olist.product_updated import Product
from olist.seller_updated import Seller

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import string
import unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup

👩🏻‍🏫 If you followed the `Decision Science` module, you already have the `olist` package installed and importable. *You can skip this section* and move to the **Data collection** section.

### (0.1) Import `olist` package`

Download a fresh version of the `olist` package:

```bash
mkdir ~/code/lewagon
cd ~/code/lewagon
git clone git@github.com:lewagon/olist.git
cd olist
git fetch
git checkout full-package
```

### (0.2) Download the datasets

- Download the datasets from Kaggle https://www.kaggle.com/olistbr/brazilian-ecommerce
- Unzip them into the `/data/csv` directory of the `olist` package:

```bash
.
├── README.md
├── data
│   └── csv
│       ├── olist_customers_dataset.csv
│       ├── olist_geolocation_dataset.csv
│       ├── olist_order_items_dataset.csv
│       ├── olist_order_payments_dataset.csv
│       ├── olist_order_reviews_dataset.csv
│       ├── olist_orders_dataset.csv
│       ├── olist_products_dataset.csv
│       ├── olist_sellers_dataset.csv
│       └── product_category_name_translation.csv
├── notebooks
├── olist
│   ├── README.md
│   ├── __init__.py
│   ├── data.py
│   ├── order.py
│   ├── product.py
│   ├── product_updated.py
│   ├── review.py
│   ├── seller.py
│   ├── seller_updated.py
│   └── utils.py
├── requirements.txt
└── setup.p
```

### (0.3) Install the `olist` package

```bash
pip install -e .
```

⚠️ Restart the kernel.

## (1) Data Collection and Preparation

👉 The way we designed the `reviews` DataFrame will help us focus on the `product categories`.

In [ ]:
reviews = Review().get_training_data()
reviews.head()

,review_id,length_review,review_score,order_id,product_category_name
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer


👉 We can retrieve the reviews from `order_reviews` within the Olist class.

In [ ]:
data = Olist().get_data()

In [ ]:
reviews_data = data['order_reviews']
reviews_data.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


🪤 We need to collect some information about the orders because:
- Back then, customers could review an order even before receiving it
- They could also rate an order that they hadn't received

In [ ]:
orders = data['orders']
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


💥 Let's merge these three datasets and apply some operations

In [ ]:
df = reviews.merge(reviews_data).merge(orders)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


🚓 Remove the reviews that were written even before receiving the order.

In [ ]:
rated_after_received = df["order_delivered_customer_date"] < df["review_creation_date"]
df = df[rated_after_received]

🚓 Remove the reviews for undelivered orders.

In [ ]:
delivered = df['order_status'] == "delivered"
df = df[delivered]

✍️ As some people fill in only the title or the body of a review, it may be a good idea to aggregate them together.

In [ ]:
df = df.dropna(subset=['review_comment_title','review_comment_message'])

df['full_review'] = df["review_comment_title"].fillna('') + " " \
            + df['review_comment_message'].fillna('')

🕵🏻 Let's focus on some columns of interest:

In [ ]:
# Focus on certain columns
columns_of_interest = ['review_id',
                       'length_review',
                       'review_score',
                       'order_id',
                       'product_category_name',
                       'full_review']
df = df[columns_of_interest]

In [ ]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review
9,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
15,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."
22,d21bbc789670eab777d27372ab9094cc,12,5,4fc44d78867142c627497b60a7e0228a,beleza_saude,Ótimo Loja nota 10
36,c92cdd7dd544a01aa35137f901669cdf,112,4,37e7875cdce5a9e5b3a692971f370151,esporte_lazer,Muito bom. Recebi exatamente o que esperava. A...
38,08c9d79ec0eba1d252e3f52f14b8e6a9,11,5,e029f708df3cc108b3264558771605c6,pet_shop,"Bom Recomendo ,"


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import string
import unidecode


def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercasing 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    sentence = unidecode.unidecode(sentence) # remove accents
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    # Advanced cleaning        
    tokenized_sentence = word_tokenize(sentence) ## tokenizing 
    stop_words = set(stopwords.words('portuguese')) ## defining stopwords
    tokenized_sentence = [w for w in tokenized_sentence 
                                  if not w in stop_words] ## remove stopwords
    lemmatized_sentence = [WordNetLemmatizer().lemmatize(word, pos = "v")  # v --> verbs
              for word in tokenized_sentence]
    cleaned_sentence = ' '.join(word for word in lemmatized_sentence)
    return cleaned_sentence

In [ ]:
df["full_review_cleaned"] = df["full_review"].apply(cleaning)
df["full_review_cleaned"]

9        recomendo aparelho eficiente site marca aparel...
15       super recomendo vendedor confiavel produto ok ...
22                                         otimo loja nota
36       bom recebi exatamente esperava demais encomend...
38                                           bom recomendo
                               ...                        
98622    nota maxima obrigado excelente atendimentobara...
98627                                             aprovado
98631    bom produto ficamos satisfeitos produto atende...
98632    nao entregue pedido bom dia unidades compradas...
98634    foto enganosa foto diferente principalmente gr...
Name: full_review_cleaned, Length: 8640, dtype: object

## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [ ]:
round(df["review_score"].value_counts(normalize = True),2)

5    0.59
4    0.16
1    0.14
3    0.07
2    0.04
Name: review_score, dtype: float64

🕵🏻‍♂️ Let's focus on these reviews...

In [ ]:
threshold = 3
bad = df["review_score"] <= threshold
df = df[bad]
df.shape

(2182, 7)

In [ ]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned
133,46d8249ea59101c7288936c3e6f155d7,119,3,f25ddb6cd62d720a5d91fff80467930a,bebes,Entrega atrasou A entrega foi dividida em duas...,entrega atrasou entrega dividida duas nao comu...
169,5f938e5f5f2e9a75710b54feeb9ea610,17,1,d9ff0185a300435402f043124f530274,eletrodomesticos,Médio A peça não serviu,medio peca nao serviu
191,c40a6b6e0181e5ec0d12cbc2e12c49d3,48,1,f3468d6a2c7586711e9952242898aad4,utilidades_domesticas,falta de produto e quebra Faltou 1 produto e o...,falta produto quebra faltou produto recebi vei...
198,06f47dacd0575e592bf6a17e922ce6cd,63,1,82348f1b5f24ac795fa9320e02c49553,esporte_lazer,não recebi ainda aqui está descrevendo como en...,nao recebi ainda aqui descrevendo entregue so ...
206,4656b8ed41c17f6be05e911e211ecd8f,21,3,7bdfa70c4049e7319a3182997e27cb95,beleza_saude,recomendo Comecei a usar agora,recomendo comecei usar agora


### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [ ]:
vectorizer = TfidfVectorizer(max_df = 0.75, max_features = 5000, ngram_range=(1,2))
vectorized_reviews = pd.DataFrame(vectorizer.fit_transform(df["full_review_cleaned"]).toarray(),
                                 columns = vectorizer.get_feature_names_out())

print(f" vectorized_reviews.shape = {vectorized_reviews.shape}")

 vectorized_reviews.shape = (2182, 5000)


### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [ ]:
n_components = 3

lda = LatentDirichletAllocation(n_components = n_components)
lda.fit(vectorized_reviews)

LatentDirichletAllocation(n_components=3)

#### Document Mixture (of Topics)

In [ ]:
document_mixture = lda.transform(vectorized_reviews)
document_mixture.shape

(2182, 3)

In [ ]:
round(pd.DataFrame(document_mixture, 
                   columns = [f"Topic {i+1}" for i in range(n_components)])
      ,2)

,Topic 1,Topic 2,Topic 3
0,0.08,0.09,0.82
1,0.67,0.20,0.13
2,0.57,0.35,0.09
3,0.46,0.46,0.08
4,0.13,0.74,0.13
...,...,...,...
2177,0.77,0.12,0.11
2178,0.07,0.87,0.07
2179,0.17,0.67,0.17
2180,0.54,0.25,0.21


👉 Let's report the most important topic for each review

In [ ]:
df["most_important_topic"] = np.argmax(document_mixture, axis = 1)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic
133,46d8249ea59101c7288936c3e6f155d7,119,3,f25ddb6cd62d720a5d91fff80467930a,bebes,Entrega atrasou A entrega foi dividida em duas...,entrega atrasou entrega dividida duas nao comu...,2
169,5f938e5f5f2e9a75710b54feeb9ea610,17,1,d9ff0185a300435402f043124f530274,eletrodomesticos,Médio A peça não serviu,medio peca nao serviu,0
191,c40a6b6e0181e5ec0d12cbc2e12c49d3,48,1,f3468d6a2c7586711e9952242898aad4,utilidades_domesticas,falta de produto e quebra Faltou 1 produto e o...,falta produto quebra faltou produto recebi vei...,0
198,06f47dacd0575e592bf6a17e922ce6cd,63,1,82348f1b5f24ac795fa9320e02c49553,esporte_lazer,não recebi ainda aqui está descrevendo como en...,nao recebi ainda aqui descrevendo entregue so ...,0
206,4656b8ed41c17f6be05e911e211ecd8f,21,3,7bdfa70c4049e7319a3182997e27cb95,beleza_saude,recomendo Comecei a usar agora,recomendo comecei usar agora,1


#### Topic Mixture (of Words)

In [ ]:
topic_mixture = pd.DataFrame(lda.components_, 
                             columns = vectorizer.get_feature_names_out())
topic_mixture.shape

(3, 5000)

In [ ]:
topic_mixture

abaixo  abaixo expectativa  abdominal    aberta  aberta rasgada  \
0  0.344601            0.334749   0.334965  0.641465        0.334634   
1  1.578281            0.881465   0.809846  1.459035        0.869869   
2  0.510370            0.334702   0.334901  0.627085        0.334503   

     aberto      abri  abri caixa  abri chamado  abri reclamacao     abrir  \
0  0.833127  0.583324    0.535585      0.336289         0.553100  0.407421   
1  1.025974  2.712356    0.795094      0.850056         0.621543  2.228424   
2  0.998847  0.336620    0.334906      0.334643         0.340251  0.338257   

   abrir embalagem   absurda   absurdo   acabado  acabamento  acabamento nao  \
0         0.444914  0.334281  0.366372  0.335081    0.336353        0.335358   
1         0.834101  0.334397  2.877594  1.318122    4.952674        0.966735   
2         0.335193  0.912201  0.642597  0.335114    1.948522        0.699883   

   acabamento ruim  acabamento tentando  acabamentos    acabei  \
0         0.335898             0.333896     0.336489  0.800285   
1         0.396270             0.333916     1.182421  0.536952   
2         1.364966             0.718968     0.334433  1.016045   

   acabei recebendo    acabou    aceita   aceitei    acerca   acessar  \
0          0.595901  0.481962  0.335551  0.334310  0.594511  0.409711   
1          0.540691  0.975914  0.492792  0.841242  0.352242  0.670977   
2          0.335289  0.548567  0.906874  0.334265  0.653764  0.396677   

   acessorios     achei  achei diferente  achei estranho  achei material  \
0    0.348146  1.874723         0.999483        0.480478        0.337270   
1    1.216149  4.265174         0.485718        0.334815        0.688003   
2    0.334966  4.393968         0.335858        0.658436        0.748152   

   achei produto      acho  acidente  acompanha  acompanhou  acontece  \
0       0.336699  1.440483  0.844812   0.562103    0.733530  0.536993   
1       1.331023  1.082171  0.334569   0.535998    0.454536  1.024117   
2       1.268612  1.692315  0.542027   0.335407    0.431652  0.746456   

   aconteceu    acordo  acredito  adaptador  adequada   adesivo  adesivos  \
0   0.351059  1.185997  0.406527   0.335243  0.335209  0.344983  0.556621   
1   2.654653  1.035443  1.565244   0.548063  0.351159  1.881320  1.018924   
2   0.344951  1.037970  0.517273   1.093544  1.023755  0.357293  0.336080   

     adorei   adquiri  adquiridas  adquirido  adquiridos    afinal   agencia  \
0  0.337193  0.421478    0.693080   0.701887    0.341672  0.335211  0.334531   
1  0.382777  0.731435    0.334848   0.992867    1.235634  0.855219  2.101534   
2  1.629927  1.050595    0.592797   0.335616    0.335269  0.334353  0.334650   

   agencia correios     agora  agora ja  agora nada  agora nao  agora ninguem  \
0          0.334739  4.721790  0.334803    0.337313   2.005469       1.410721   
1          1.125061  7.378473  0.892945    2.731236   2.382924       0.514315   
2          0.334796  1.234884  0.361865    0.334940   0.637342       0.349086   

   agora resposta  agora so  agradavel  agradeco   agradou      agua  \
0        0.856451  0.506231   0.570009  1.330682  0.946778  0.340718   
1        0.334242  0.671888   0.334829  0.340380  0.334057  2.038525   
2        0.345976  0.339506   1.008062  0.334351  0.334109  0.335445   

   aguardando  aguardando ainda  aguardando chegar  aguardando contato  \
0    3.898455          0.521997           0.334428            0.335245   
1    7.921563          0.694425           0.793896            0.365988   
2    3.263094          0.376501           0.335198            0.852208   

   aguardando desfecho  aguardando entrega  aguardando nao  aguardando outro  \
0             0.846657            0.341478        0.700931          0.700950   
1             0.342054            2.806623        0.345363          0.335228   
2             0.334165            0.348059        0.548187          0.694690   

   aguardando posicionamento  aguardando resposta  aguardando re

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [ ]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [ ]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))
        

🕵🏻‍♂️ Print the topics with their topwords:

In [ ]:
print_topics(vectorizer, lda, topwords = 5)

--------------------
Topic 0:
[('produto', 23.94), ('nao', 19.0), ('veio', 18.29), ('recebi', 17.52), ('comprei', 15.47)]
--------------------
Topic 1:
[('produto', 74.89), ('nao', 68.28), ('bom', 55.19), ('recebi', 47.42), ('nao recebi', 33.32)]
--------------------
Topic 2:
[('nao', 32.79), ('produto', 24.96), ('entregue', 14.78), ('produto nao', 13.8), ('veio', 13.51)]


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _producto = product_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [ ]:
topic_word_mixture = [topic_word(vectorizer, lda, topic, topwords = 5, with_weights = False)
                      for topic in lda.components_]
topic_word_mixture

[['produto', 'nao', 'veio', 'recebi', 'comprei'],
 ['produto', 'nao', 'bom', 'recebi', 'nao recebi'],
 ['nao', 'produto', 'entregue', 'produto nao', 'veio']]

In [ ]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

In [ ]:
df[["review_id", 
        "review_score", 
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head(3)

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
133,46d8249ea59101c7288936c3e6f155d7,3,bebes,entrega atrasou entrega dividida duas nao comu...,2,"[nao, produto, entregue, produto nao, veio]"
169,5f938e5f5f2e9a75710b54feeb9ea610,1,eletrodomesticos,medio peca nao serviu,0,"[produto, nao, veio, recebi, comprei]"
191,c40a6b6e0181e5ec0d12cbc2e12c49d3,1,utilidades_domesticas,falta produto quebra faltou produto recebi vei...,0,"[produto, nao, veio, recebi, comprei]"


## (3.4) Pipeline Tf-Idf and LDA

In [ ]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [ ]:
%%time
# Parameters
max_df = 0.75
max_features = 5000
ngram_range = (1,2)

# Pipeline Vectorizer + LDA
pipeline = make_pipeline(
    TfidfVectorizer(max_df = max_df,
                    max_features = max_features,
                    ngram_range = ngram_range),
    LatentDirichletAllocation(n_components = n_components)
)

# Fit the pipeline on the cleaned texts
pipeline.fit(df["full_review_cleaned"])

CPU times: user 5.15 s, sys: 77 ms, total: 5.23 s
Wall time: 6.1 s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.75, max_features=5000,
                                 ngram_range=(1, 2))),
                ('latentdirichletallocation',
                 LatentDirichletAllocation(n_components=3))])

💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [ ]:
pipeline._final_estimator

LatentDirichletAllocation(n_components=3)

In [ ]:
pipeline._final_estimator.components_

array([[0.3828966 , 0.33424687, 0.33419884, ..., 0.33485356, 0.53853056,
        0.49823753],
       [1.58236185, 0.88118754, 0.33424263, ..., 0.57183943, 0.80394727,
        0.68673599],
       [0.46799278, 0.3354807 , 0.81127032, ..., 0.60235224, 0.33603871,
        0.87298567]])

**Document Mixture** with the Pipeline:

In [ ]:
# Transform the original cleaned texts with the pipeline
# Indeed, there is no need to get the vectorized texts first since it's done through the Pipeline
document_mixture = pipeline.transform(df["full_review_cleaned"]) 
document_mixture.shape 

(2182, 3)

**Topic Mixture** with the Pipeline:

In [ ]:
topic_mixture = pd.DataFrame(pipeline._final_estimator.components_)
topic_mixture.shape

(3, 5000)

In [ ]:
topic_mixture

0         1         2         3         4         5         6     \
0  0.382897  0.334247  0.334199  1.769390  0.868942  1.379610  0.715796   
1  1.582362  0.881188  0.334243  0.335013  0.334103  1.126317  2.573512   
2  0.467993  0.335481  0.811270  0.623181  0.335961  0.352022  0.342992   

       7         8         9         10        11        12        13    \
0  0.352000  0.334462  0.334136  0.782741  0.788431  0.657060  2.833753   
1  0.971433  0.852022  0.846040  1.423173  0.340716  0.586211  0.377804   
2  0.342152  0.334504  0.334719  0.768187  0.485061  0.337607  0.675005   

       14        15        16        17        18        19        20    \
0  0.334999  0.349037  0.338541  0.334061  0.333928  0.616350  0.686165   
1  0.337823  0.377099  0.340896  0.334175  0.354960  0.335233  1.318461   
2  1.315496  6.511413  1.322538  1.428898  0.697893  0.901761  0.348656   

       21        22        23        24        25        26        27    \
0  0.799813  0.502701  0.625050  0.565923  0.673564  0.335771  1.204334   
1  0.337737  1.168482  0.774626  0.605700  0.590601  0.807385  0.356077   
2  0.334330  0.335259  0.335541  0.338194  0.336353  0.334210  0.338850   

       28        29        30        31        32        33        34    \
0  4.158308  0.702619  0.804714  1.096865  0.546805  0.948003  0.334357   
1  1.602406  0.334937  0.334516  0.334457  0.854285  1.763725  1.052220   
2  4.773151  0.783503  0.334500  0.342103  1.535244  1.503242  0.334831   

       35        36        37        38        39        40        41    \
0  0.334532  0.336255  1.334808  2.267268  0.949948  1.254628  1.206076   
1  0.764561  0.777656  0.637491  0.746360  1.728428  0.352026  0.435597   
2  0.334415  0.505805  0.335267  0.337035  0.581034  0.882391  0.335177   

       42        43        44        45        46        47        48    \
0  0.937789  0.737485  0.480545  1.428963  0.860897  0.951727  0.614158   
1  0.436816  1.378240  0.339173  0.335080  0.745148  0.334380  1.080741   
2  0.335518  0.467872  1.091907  0.585853  0.597462  0.334617  0.335470   

       49        50        51        52         53        54        55    \
0  0.916620  0.598395  2.102245  1.126025   1.738713  0.700626  0.334295   
1  0.660133  0.591306  0.334210  0.334213  11.218164  0.340979  2.734138   
2  0.335820  0.335081  0.334259  0.334357   0.378269  0.548008  0.335056   

       56        57        58        59        60        61        62    \
0  0.336049  1.029500  0.358397  0.392361  0.942209  0.818156  0.759442   
1  4.354008  0.908838  0.843574  0.721223  0.572561  0.335222  0.360353   
2  0.335678  0.335785  0.334699  0.404042  0.398130  0.852034  0.495150   

       63        64        65        66        67        68        69    \
0  1.456965  5.533555  0.507500  0.719278  0.885064  0.343659  0.395253   
1  0.366496  9.193669  0.750356  0.409361  0.334182  0.844637  2.764929   
2  0.891227  0.355888  0.335068  0.334884  0.334196  0.334580  0.335979   

       70        71        72        73        74        75        76    \
0  0.335036  0.687980  0.665734  0.334195  1.779289  0.356804  0.334613   
1  0.683418  0.706781  0.617388  1.829636  0.812758  0.927891  1.034813   
2  0.576027  0.336108  0.335546  0.334337  0.335726  0.335079  0.334976   

       77        78        79        80        81        82        83    \
0  0.334096  4.937534  1.006612  1.092907  0.334703  0.583074  0.469637   
1  1.185634  4.605504  0.334322  0.334454  0.408927  0.557923  1.712015   
2  0.334905  1.260010  0.334269  0.334738  1.547766  0.334760  0.344540   

       84        85        86        87        88        89         90    \
0  1.527534  1.136324  0.333877  0.334587  1.494535  0.690245   7.261098   
1  1.593753  0.334135  0.334556  0.339586  0.366987  0.819946  18.028996   
2  0.370777  0.338217  0.715439  0.671562  0.531223  0.335735   1.028935   

       91        92        93        94         95        96        97    \
0  1.55551

## (4) Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [ ]:
# Product categories by performance - look at the count, mean, median and std
# $CHALLENGIFY_BEGIN
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })
# $CHALLENGIFY_END

# Removing products which were sold less than a certain times for the analysis
# $CHALLENGIFY_BEGIN
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]
# $CHALLENGIFY_END


# Sorting the product categories by performance
# $CHALLENGIFY_BEGIN
product_categories = product_categories.sort_values(by = [('review_score', 'mean'), 
                                                          ('review_score', 'std')], 
                                                    ascending = [False, True])
# $CHALLENGIFY_END

product_categories

review_score                           
                              count      mean median       std
product_category_name                                         
moveis_decoracao                129  1.852713    2.0  0.893560
esporte_lazer                   118  1.822034    1.0  0.921086
relogios_presentes              243  1.818930    1.0  0.900030
bebes                            80  1.812500    1.0  0.915323
utilidades_domesticas           144  1.784722    1.0  0.870360
cama_mesa_banho                 256  1.777344    1.0  0.890712
brinquedos                       52  1.750000    1.0  0.882843
beleza_saude                    170  1.747059    1.0  0.917043
automotivo                      104  1.673077    1.0  0.841200
telefonia                       100  1.670000    1.0  0.841535
informatica_acessorios          165  1.666667    1.0  0.871873

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [ ]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                              count      mean median       std
product_category_name                                         
beleza_saude                    170  1.747059    1.0  0.917043
informatica_acessorios          165  1.666667    1.0  0.871873
automotivo                      104  1.673077    1.0  0.841200
telefonia                       100  1.670000    1.0  0.841535
brinquedos                       52  1.750000    1.0  0.882843

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [ ]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id", 
                        "review_score", 
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
206,4656b8ed41c17f6be05e911e211ecd8f,3,beleza_saude,recomendo comecei usar agora,1,"[produto, nao, bom, recebi, nao recebi]"
212,6b341682ab39af9fa00d72d7388c903b,3,automotivo,nota comprar produto correto capa interno ser ...,1,"[produto, nao, bom, recebi, nao recebi]"
345,ef9ef383ff6625e8e118db388ad3420d,3,beleza_saude,insatisfeito site produto parece ter melhor qu...,1,"[produto, nao, bom, recebi, nao recebi]"
444,320ae91d4c5cb8164c168fc68392a5fb,1,informatica_acessorios,produto pirata instalei cartucho tinta impress...,2,"[nao, produto, entregue, produto nao, veio]"
707,bde05133984518116b3d77d994e375d3,2,informatica_acessorios,produto incorreto produto nao medidas corretas...,1,"[produto, nao, bom, recebi, nao recebi]"
...,...,...,...,...,...,...
98048,142e8caaa4b16240c5fc1118684c80fa,3,beleza_saude,nunca demorou tanto chega produtos,0,"[produto, nao, veio, recebi, comprei]"
98376,623b07b8ac7a4947eb6a88f2a4def222,1,beleza_saude,nao recomendo produto veio estragado ruim,2,"[nao, produto, entregue, produto nao, veio]"
98401,44380245bf1a49875542ea0e7f216986,1,informatica_acessorios,nao recebi produto aind assim receber produto ...,0,"[produto, nao, veio, recebi, comprei]"
98549,b01ab0ba0ad74ab16c994956e5cf20e6,3,beleza_saude,super recomendo otimo,0,"[produto, nao, veio, recebi, comprei]"


### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [ ]:
worst_products_reviews["most_important_topic"].value_counts()

1    300
2    163
0    128
Name: most_important_topic, dtype: int64

In [ ]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

[1, 2, 0]

In [ ]:
[topic_word_mixture[i] for i in bad_frequency]

[['produto', 'nao', 'bom', 'recebi', 'nao recebi'],
 ['nao', 'produto', 'entregue', 'produto nao', 'veio'],
 ['produto', 'nao', 'veio', 'recebi', 'comprei']]

## (5) Sellers...

* What kind of products were sold by the worst sellers ?
* What are the main reviews for the worst sellers ?

### (5.1) Worst Sellers

In [ ]:
sellers = Seller().get_training_data()
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'share_of_one_stars', 'share_of_five_stars', 'review_score',
       'cost_of_reviews', 'n_orders', 'quantity', 'quantity_per_order',
       'sales', 'revenues', 'profits'],
      dtype='object')

👇 Select the 10 worst sellers and store them into a variable called `worst_sellers`

In [ ]:
worst_sellers = sellers[["seller_id", "review_score", "profits"]].sort_values(
    by = "profits", 
    ascending = True).head(10)
worst_sellers

,seller_id,review_score,profits
769,6560211a19b47992c3666cc44a7e94c0,3.937093,-21519.517
2358,4a3ca9315b744ce9f8e9374361493884,3.827873,-17752.708
1357,cc419e0650a3c5ba77189a1882b7556a,4.077586,-17581.158
945,ea8482cd71df3c1969d7b9473ff13abc,3.997371,-15542.248
315,8b321bb669392f5163d04c59e235e066,4.074547,-12886.431
453,1f50f920176fa81dab994f9023523100,4.130342,-9926.079
1213,d2374cbcbb3ca4ab1086534108cc3ab7,3.751880,-9017.608
1132,7c67e1448b00f6e969d365cea6b010ab,3.488798,-8017.611
2023,cca3071e3e9bb7d12640c9fbe2301306,3.846369,-7379.011
2687,1835b56ce799e6a4dc4eddc053f04066,3.637441,-7135.579


### (5.2) Products sold by the worst sellers

In [ ]:
products = Product().get_training_data() [["product_id", "category"]]
products

,product_id,category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumery
...,...,...
31479,13b25797ad1e6d6c7cd3cbeb35f8ee7a,furniture_mattress_and_upholstery
31480,dc36a7859b743d8610a2bbbaea26ece9,furniture_mattress_and_upholstery
31481,107fde0930956120d1e13dd1062fbb46,furniture_mattress_and_upholstery
31482,726b4e18f00255e2e63491bcba3f60b8,furniture_mattress_and_upholstery


❓ What are the types of products sold by the worst sellers ❓

In [ ]:
sellers_product_category = data["order_items"].merge(products, 
                                             on = "product_id", how = "left")[["seller_id", "category"]]

sellers_product_category

,seller_id,category
0,48436dade18ac8b2bce089ec2a041202,cool_stuff
1,dd7ddc04e1b6c2c614352b383efe2d36,pet_shop
2,5b51032eddd242adc84c38acab88f23d,furniture_decor
3,9d7a1d34a5052409006425275ba1c2b4,perfumery
4,df560393f3a51e74553ab94004ba5c87,garden_tools
...,...,...
112645,b8bc237ba3788b23da09c0f1f3a3288c,housewares
112646,f3c38ab652836d21de61fb8314b69182,computers_accessories
112647,c3cfdc648177fdbbbb35635a37472c53,sports_leisure
112648,2b3e4a2a3ea8e01938cabda2a3e5cc79,computers_accessories


In [ ]:
sellers_product_category.groupby("seller_id").count()

,category
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,3
001cca7ae9ae17fb1caed9dfb1094831,239
001e6ad469a905060d959994f1b41e4f,0
002100f778ceb8431b7a1020ff7ab48f,55
003554e2dce176b5555353e4f3555ac8,0
...,...
ffcfefa19b08742c5d315f2791395ee5,0
ffdd9f82b9a447f6f8d4b91554cc7dd3,20
ffeee66ac5d5a62fe688b9d26f83f534,14


### (5.3) Categories and topics for the worst sellers

🎁 Here are some useful functions:
- `focus_seller(seller_id)` to show the product categories sold by a seller
- `bad_reviews_seller` to show to topwords of the most frequent topic for one seller

In [ ]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [ ]:
bad_reviews_sellers = df.merge(data["order_items"])
bad_reviews_sellers.head(3)

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic,most_important_words,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,46d8249ea59101c7288936c3e6f155d7,119,3,f25ddb6cd62d720a5d91fff80467930a,bebes,Entrega atrasou A entrega foi dividida em duas...,entrega atrasou entrega dividida duas nao comu...,2,"[nao, produto, entregue, produto nao, veio]",1,41cd333a4af5e36df81fc0b83ccf8e2d,5dceca129747e92ff8ef7a997dc4f8ca,2018-01-18 13:29:24,388.0,32.13
1,5f938e5f5f2e9a75710b54feeb9ea610,17,1,d9ff0185a300435402f043124f530274,eletrodomesticos,Médio A peça não serviu,medio peca nao serviu,0,"[produto, nao, veio, recebi, comprei]",1,082175ee534338e4fd967076a6f848e5,d91fb3b7d041e83b64a00a3edfb37e4f,2018-05-10 16:54:16,52.0,18.24
2,c40a6b6e0181e5ec0d12cbc2e12c49d3,48,1,f3468d6a2c7586711e9952242898aad4,utilidades_domesticas,falta de produto e quebra Faltou 1 produto e o...,falta produto quebra faltou produto recebi vei...,0,"[produto, nao, veio, recebi, comprei]",1,31a89a7044c5b133cf646404a4e86022,955fee9216a65b617aa5c0531780ce60,2018-05-17 20:56:02,150.0,17.64


In [ ]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
    return topic_word_mixture[most_frequent_topic_seller]

❓For each of these worst sellers, show the most frequent product categories and words ❓

In [ ]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))
    

--------------------------------------------------------------------------------
Focusing on the seller #6560211a19b47992c3666cc44a7e94c0...
seller_id                         category                 
6560211a19b47992c3666cc44a7e94c0  watches_gifts                1624
                                  fashion_bags_accessories      340
                                  audio                          32
                                  perfumery                      13
                                  computers_accessories          12
                                  sports_leisure                  7
                                  construction_tools_safety       1
dtype: int64
['produto', 'nao', 'bom', 'recebi', 'nao recebi']
--------------------------------------------------------------------------------
Focusing on the seller #4a3ca9315b744ce9f8e9374361493884...
seller_id                         category       
4a3ca9315b744ce9f8e9374361493884  bed_bath_table     1566
           

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this new "expertise" with Decision Science

💾 Don't forget to `git add / commit / push`